<a href="https://colab.research.google.com/github/20911357Pinyaphat/20911357-smart-finance-assistant/blob/main/Projects/Smart-assistance-for-girlys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 💖 **Budget Babe: Smart Finance Assistant**

Welcome to Budget Babe — your friendly, empowering finance assistant designed to Help users in Australia track recurring subscriptions, understand their spending habits, and reach savings goals with confidence. Automatically detect repeating transactions from CSV data, categorize services, and identify overlaps or excessive spending. Empower users to make smart, guilt-free cuts and celebrate every step toward financial freedom..

🎯 **What This App Is For:**
- Load real data from `sample_transactions.csv` that includes popular Australian subscriptions and they can also upload or manually enter their subscriptions.
- Automatically categorize each subscription (e.g., Streaming, Music, Food, Utilities)
- Help users identify duplicate or overlapping services they might want to cancel
- Let users name themselves and their chatbot for a personalized experience
- Allow users to set a savings goal name (e.g., "Trip to Bali", "New Laptop") and track progress
- Show monthly and yearly spending reports
- Display infographics and graphs to visualize spending by category
- Provide friendly, girly, and empowering chatbot advice to help users make smart decisions

✨ **Why We're Building This:**
Budgeting should feel joyful, intuitive, and supportive — not boring or stressful. Budget Babe is here to:
- Make subscription tracking effortless
- Help users save money without guilt
- Celebrate every step toward financial freedom

Let’s glow up your budget together 💅
'''


# *💖 Budget Babe: Six-Step Development Methodology*
**STEP 1**: Understand the Problem
🎯 Define Your Finance Problem
Help young professionals track recurring subscriptions (e.g., Netflix, Spotify, gym). Automatically detect repeating transactions from CSV data, calculate monthly and yearly costs, and alert users when subscriptions exceed their budget or overlap unnecessarily.


**STEP 2**: Identify Inputs and Outputs
📥 Define Your Data Flow

Inputs:
- CSV file with transaction data (Columns: Date, Amount, Category, Description)
- User-defined savings goal and current savings
- user-selected subscriptions via checkbox entrys or manual entry

Outputs:
- categorized subscription list
- month and Annual subscription totals
- Alerts for budget breaches or duplicate services
- saving progress
- chatbot advice


**STEP 3**: Work the Problem by Hand
manual conculations
- total monthly spend $16.99 + $50 = $66.99
- overbudgeting

**STEP 4**: Write Pseudocode
📝 Plan Your Solution Logic
plaintext
FUNCTION analyze_spending_data(csv_file):
1. Load and clean data
2. Auto-detect categories from keywords
3. Identify subscriptions and group by category
4. Calculate monthly and yearly totals
5. Compare against savings goal
6. Generate cut suggestions and feedback
RETURN summaries, suggestions, sparkle score

**STEP 5**: Convert to Python (in section below)



In [44]:
# 💖 Install and import
!pip install hands_on_ai pandas gradio --quiet

import hands_on_ai.chat
import pandas as pd
import gradio as gr
from hands_on_ai.chat import get_response
print(get_response("Hello Budget Babe!"))


import os
from getpass import getpass
# Configure hands-on-ai server connection
os.environ['HANDS_ON_AI_SERVER'] = 'https://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'llama3.2'
os.environ['HANDS_ON_AI_API_KEY'] = getpass('Enter your API key: ')

# 💖 Load and clean data
csv_url = "https://raw.githubusercontent.com/20911357Pinyaphat/smart-finance-assistant/main/data/sample_transactions.csv"
df = pd.read_csv(csv_url)
df["Description"] = df["Description"].str.strip().str.title()

df["Amount"] = pd.to_numeric(df["Amount"].replace(r'[\$,]', '', regex=True))
csv_upload = gr.File(label="Upload Your Subscription CSV", file_types=[".csv"])
upload_status = gr.Textbox(label="Upload Status", lines=2)

original_df = df.copy()
# 💖 Categorize subscriptions
# 💖 Keyword-based categorization
category_keywords = {
    "Music": ["spotify", "apple music", "tidal", "youtube music", "amazon music", "deezer", "pandora", "soundcloud"],
    "Movies & TV": ["netflix", "disney", "stan", "binge", "foxtel", "paramount", "prime video", "youtube premium", "kayo", "apple tv"],
    "Fitness": ["fitness", "gym", "classpass", "les mills", "centr", "sweat", "headspace", "calm", "revo", "f45", "virgin active"],
    "Reading": ["audible", "kindle", "scribd", "masterclass", "coursera", "blinkist", "medium", "guardian", "economist", "nyt"],
    "Food": ["hellofresh", "marley spoon", "dinnerly", "coffee", "cheese", "wine", "beer", "snack", "vegan box", "goodnessme"],
    "Shopping": ["amazon prime", "ebay plus", "iconic", "coles plus", "woolworths", "catch", "kogan", "jb hi-fi", "big w"],
    "Beauty & Lifestyle": ["beauty", "retreat", "bellabox", "lustbox", "therabox", "mindbox", "man box", "eco box", "local drop"],
    "Internet & Phone": ["telstra", "optus", "vodafone", "nbn", "internet", "mobile", "broadband", "exetel", "iinet", "amaysim", "belong"],
    "Software": ["canva", "adobe", "microsoft", "notion", "grammarly", "dropbox", "zoom", "todoist", "google one", "evernote"],
    "Gaming": ["xbox", "playstation", "nintendo", "steam", "ea play", "ubisoft", "arcade", "crunchyroll", "geforce"]
}
manual_services = []

# 💖 Categorize subscriptions
def categorize_subscriptions(description):
    desc = description.lower()
    for category, keywords in category_keywords.items():
        if any(keyword in desc for keyword in keywords):
            return category
    return "Other"

# 💖 Apply categorization
df["Category"] = df["Description"].apply(categorize_subscriptions)
df = df.dropna(subset=["Description", "Category"])

# 💖 Build category map
category_map = {}
for _, row in df.iterrows():
    category_map.setdefault(row["Category"], []).append(row["Description"])



# 💖 Global state
user_info = {
    "name": "",
    "bot": "",
    "goal": "",
    "target": 0.0
}

checkbox_groups = {}  # category → gr.CheckboxGroup component


# 💖 Helper functions
def save_user_info(name, bot_name, goal_name, goal_amount, current_savings):
    try:
        user_info["name"] = name.strip().title()
        user_info["bot"] = bot_name.strip().title()
        user_info["goal"] = goal_name.strip().title()
        user_info["target"] = float(goal_amount) if goal_amount else 0.0
        user_info["saved"] = float(current_savings) if current_savings else 0.0

        return f"Hi {user_info['name']}! Your budgeting assistant '{user_info['bot']}' is ready 💖\n" \
               f"Goal: {user_info['goal']} (${user_info['target']:.2f})\n" \
               f"Current savings: ${user_info['saved']:.2f}"
    except:
        return "⚠️ Please enter valid numbers for your goal and current savings 💅"

def auto_detect_category(description):
    desc = description.lower()
    for category, keywords in category_keywords.items():
        if any(keyword in desc for keyword in keywords):
            return category
    return "Other"


def collect_selected(*groups):
    selected = []
    for group in groups:
        if isinstance(group, list):
            selected.extend(group)
    return selected



def calculate_spending(selected_services):
    all_services = [s.strip().title() for s in selected_services + manual_services]
    selected_df = df[df["Description"].isin(all_services)]

    if selected_df.empty:
        return "⚠️ No matching subscriptions found. Try selecting from the list or adding a custom one 💅"

    total = abs(selected_df["Amount"].sum())
    yearly = total * 12
    breakdown = selected_df.groupby("Category")["Amount"].sum().abs().to_dict()

    result = f"💖 Total Monthly Spend: ${total:.2f}\n"
    result += f"📅 Estimated Yearly Spend: ${yearly:.2f}\n\n"
    for cat, amt in breakdown.items():
        result += f"• {cat}: ${amt:.2f}\n"

    # 🎯 Goal logic
    target = user_info.get("target", 0)

    if target > 0:
        saved = user_info.get("saved", 0)
        monthly_savings_needed = target / 12
        current_savings = monthly_savings_needed - total
        remaining_to_save = max(0, target - saved)

        result += f"\n🎯 Monthly Savings Needed: ${monthly_savings_needed:.2f}\n"
        result += f"💰 Current Monthly Savings: ${current_savings:.2f}\n"
        result += f"🔄 Remaining to Reach Goal: ${remaining_to_save:.2f}\n"

        if current_savings < 0:
            result += "💸 You're overspending — consider trimming subscriptions!"
        elif current_savings == 0:
            result += "⚖️ You're breaking even — try cutting a few to save!"
        else:
            result += "✅ You're saving — keep sparkling!"

    return result


def import_manual_csv(file):
    try:
        user_df = pd.read_csv(file.name)
        required_cols = {"Date", "Amount", "Description"}
        if not required_cols.issubset(user_df.columns):
            return "⚠️ CSV must include Date, Amount, and Description columns."

        # Clean and format
        user_df["Description"] = user_df["Description"].str.strip().str.title()
        user_df["Amount"] = pd.to_numeric(user_df["Amount"], errors="coerce").abs()
        user_df["Date"] = pd.to_datetime(user_df["Date"], errors="coerce")

        # Auto-detect missing categories
        if "Category" not in user_df.columns:
            user_df["Category"] = user_df["Description"].apply(auto_detect_category)
        else:
            user_df["Category"] = user_df["Category"].fillna("").replace("", "Other")
            user_df["Category"] = user_df.apply(
                lambda row: auto_detect_category(row["Description"]) if row["Category"] == "Other" else row["Category"],
                axis=1
            )

        user_df = user_df.dropna(subset=["Description", "Amount", "Category"])

        # Merge into main df
        global df, manual_services, category_map
        df = pd.concat([df, user_df], ignore_index=True)

        # Track manual services
        manual_services.extend(user_df["Description"].tolist())

        # Update category map
        for _, row in user_df.iterrows():
            category_map.setdefault(row["Category"], []).append(row["Description"])

        return f"✅ Imported {len(user_df)} subscriptions with auto-detected categories!"
    except Exception as e:
        return f"❌ Error importing CSV: {str(e)}"




def add_custom_subscription(name, price, category):
    try:
        price = float(str(price).replace("$", "").replace(",", "").strip())
        name = name.strip().title()
        new_row = pd.DataFrame([{
            "Date": pd.Timestamp.today().strftime("%Y-%m-%d"),
            "Amount": abs(price),
            "Description": name,
            "Category": category
        }])
        global df
        df = pd.concat([df, new_row], ignore_index=True)

        # Save to manual list
        manual_services.append(name)

        # Update category map
        category_map.setdefault(category, []).append(name)

        return f"✅ Added: {name} (${price:.2f}) under {category}"
    except:
        return "⚠️ Please enter a valid price."

manual_output = gr.Textbox(label="Manual Subscriptions", lines=4)

def show_manual_subs():
    return "\n".join(manual_services) if manual_services else "None yet 💅"
import re


from difflib import get_close_matches

def rag(user_message, selected_df):
    message = user_message.lower()

    # 📊 Average cost by category — still uses selected_df
    if "average" in message or "cost" in message:
        breakdown = selected_df.groupby("Category")["Amount"].mean()
        if breakdown.empty:
            return "⚠️ No matching subscriptions found 💅"
        response = "📊 Average Monthly Cost by Category:\n"
        for cat, amt in breakdown.items():
            response += f"• {cat}: ${abs(amt):.2f}\n"
        return response

    # 🔍 Dynamic comparison — now uses full df
    elif "compare" in message:
        all_services = df["Description"].dropna().unique().tolist()
        all_services_lower = [s.lower() for s in all_services]
        words = message.split()
        found = [s for s in all_services if s.lower() in message]

        # Fallback to fuzzy match if direct match fails
        if len(found) < 2:
            found = get_close_matches(message, all_services_lower, n=5, cutoff=0.6)
            found = [s.title() for s in found if s.title() in all_services]

        if len(found) >= 2:
            compare_df = df[df["Description"].isin(found)]
            grouped = compare_df.groupby("Description")["Amount"].sum()
            if grouped.empty:
                return "⚠️ No matching services found 💅"
            response = "🔍 Comparison:\n"
            for name, amt in grouped.items():
                response += f"• {name}: ${abs(amt):.2f}\n"
            return response

        return "⚠️ I couldn’t find enough matching services to compare. Try using exact names 💅"

    # 🧠 Fallback
    return "🤖 I'm not sure what you're asking. Try asking about spending, cuts, goals, or comparisons!"




from hands_on_ai.chat import get_response

def budget_babe_chatbot(chat_input, chat_history, *selected_services_groups):
    print("🧠 Budget Babe AI is running...")

    selected_services = collect_selected(*selected_services_groups)
    all_services = [s.strip().title() for s in selected_services + manual_services]
    selected_df = df[df["Description"].isin(all_services)]

    context = f"""
You are Budget Babe, a friendly, girly, and empowering finance assistant.
The user has selected these subscriptions: {', '.join(all_services)}.
Their message: "{chat_input}"
Respond with warm, practical advice. Include spending tips, cut suggestions, and sparkle encouragement.
"""

    try:
        response = get_response(context)
        if not response or "@" in response or "🤔" in response:
            fallback = rag(chat_input, selected_df)
            response = f"💡 Here's a fallback tip:\n\n{fallback}"
    except Exception as e:
        response = f"⚠️ AI error: {str(e)}"

    # Append messages in correct format
    chat_history.append({"role": "user", "content": chat_input})
    chat_history.append({"role": "assistant", "content": response})
    return chat_history


def update_savings_progress():
    selected_services = collect_selected()  # ✅ Pulls from global or stored state
    all_services = [s.strip().title() for s in selected_services + manual_services]
    selected_df = df[df["Description"].isin(all_services)]

    total = abs(selected_df["Amount"].sum())
    target = user_info.get("target", 0)
    saved = user_info.get("saved", 0)

    if target == 0:
        return 0, "⚠️ Set a savings goal to track progress."

    monthly_savings = max(0, (target / 12) - total)
    projected_total = saved + monthly_savings
    progress = int(min(100, (projected_total / target) * 100))

    # 🎉 Emoji-based milestone badge
    if progress >= 100:
        badge = "🎉 **You did it! Goal reached – confetti time!**"
    elif progress >= 80:
        badge = "💖 **Almost there – keep sparkling!**"
    elif progress >= 50:
        badge = "✨ **Halfway there – keep going!**"
    else:
        badge = "🌱 **Just getting started – every cut counts!**"

    return progress, badge



def generate_report(selected_services):
    selected_df = df[df["Description"].isin(selected_services)]

    if selected_df.empty:
        return "⚠️ No subscriptions selected. Pick a few and try again 💅"

    total = abs(selected_df["Amount"].sum())
    top = selected_df.groupby("Category")["Amount"].sum().sort_values(ascending=False)

    report = f"📋 Monthly Report\n\n💖 Total Spend: ${total:.2f}\n\n🔝 Top Categories:\n"
    for cat, amt in top.items():
        report += f"• {cat}: ${amt:.2f}\n"

    return report


def compare_services_table(selected_services):
    all_services = selected_services + manual_services
    selected_df = df[df["Description"].isin(all_services)]

    if selected_df.empty:
        return "⚠️ No matching subscriptions found 💅"

    # 🧁 Emoji map for categories
    emoji_map = {
        "Movies & TV": "🎬",
        "Music": "🎵",
        "Food": "🍔",
        "Fitness": "💪",
        "Shopping": "🛍️",
        "Internet & Phone": "📶"
    }

    # ✨ Sort by category then by cost
    sorted_df = selected_df.copy()
    sorted_df["Monthly"] = sorted_df["Amount"]
    sorted_df["Yearly"] = sorted_df["Amount"] * 12
    sorted_df = sorted_df.sort_values(by=["Category", "Monthly"], ascending=[True, False])

    # 💖 Build markdown table
    table = "| Service | Monthly | Yearly | Category |\n"
    table += "|--------|---------|--------|----------|\n"

    for _, row in sorted_df.iterrows():
        name = row["Description"]
        monthly = abs(row["Monthly"])
        yearly = abs(row["Yearly"])
        category = row["Category"]
        emoji = emoji_map.get(category, "")
        highlight = "**" if monthly > 20 else ""
        table += f"| {name} | {highlight}${monthly:.2f}{highlight} | ${yearly:.2f} | {emoji} {category} |\n"

    return table

# 💖 Theme
sparkle_theme = gr.themes.Base(
    primary_hue="pink",
    secondary_hue="rose",
    font="Quicksand",
    spacing_size="lg",
    radius_size="lg"
)

# 💖 Launch App
with gr.Blocks(theme=sparkle_theme) as app:
    gr.HTML("""
    <link href="https://fonts.googleapis.com/css2?family=Quicksand&display=swap" rel="stylesheet">
    <style>
    body {
      background: linear-gradient(135deg, #ffe6f0, #fff0f5);
      font-family: 'Quicksand', sans-serif;
    }
    h2 {
      color: #d63384;
      font-size: 2em;
      text-align: center;
      margin-top: 20px;
    }
    button {
      background-color: #ff69b4 !important;
      color: white !important;
      border-radius: 12px !important;
      font-weight: bold;
    }
    input, textarea, select {
      border-radius: 10px !important;
      border: 2px solid #ffb6c1 !important;
    }
    </style>
    """)
    gr.HTML("""
    <style>
    input[type="range"] {
      height: 12px;
      border-radius: 6px;
      background: linear-gradient(90deg, #ff69b4, #ffb6c1, #ffc0cb);
      box-shadow: inset 0 0 5px rgba(255, 105, 180, 0.5);
    }

    input[type="range"]::-webkit-slider-thumb {
      background: radial-gradient(circle, #fff0f5 30%, #ff69b4 70%);
      border: 2px solid #ff69b4;
      border-radius: 50%;
      width: 20px;
      height: 20px;
      cursor: pointer;
    }

    input[type="range"]::-moz-range-thumb {
      background: radial-gradient(circle, #fff0f5 30%, #ff69b4 70%);
      border: 2px solid #ff69b4;
      border-radius: 50%;
      width: 20px;
      height: 20px;
      cursor: pointer;
    }
    </style>
    """)



    gr.Markdown("<h2>💖 Budget Babe: Smart Finance Assistant</h2>")
    chat_history = gr.State([])

    # 💖 Welcome Section
    with gr.Row():
        name_input = gr.Textbox(label="Your name")
        bot_name_input = gr.Textbox(label="Name your chatbot")
        goal_input = gr.Textbox(label="Your goal name")
        current_savings_input = gr. Textbox(label="Current savings ($)")
        goal_amount_input = gr.Textbox(label="Target amount ($)")
        welcome_output = gr.Textbox(label="Welcome Message", lines=3)
        gr.Button("Start").click(
            fn=save_user_info,
            inputs=[name_input, bot_name_input, goal_input, goal_amount_input, current_savings_input],
            outputs=welcome_output
        )

    # 💖 Subscription Picker
    gr.Markdown("### Select Your Subscriptions")
    with gr.Row():
        for cat, services in category_map.items():
            with gr.Column():
                checkbox_groups[cat] = gr.CheckboxGroup(label=cat, choices=services)

    # 💖 Add Custom Subscription
    gr.Markdown("### Add Your Own Subscription")
    with gr.Row():
        custom_name = gr.Textbox(label="Service Name")
        custom_price = gr.Textbox(label="Monthly Price ($)")
        custom_category = gr.Dropdown(label="Category", choices=list(category_map.keys()))
    add_output = gr.Textbox(label="Status", lines=2)
    gr.Button("Add Subscription").click(
        fn=add_custom_subscription,
        inputs=[custom_name, custom_price, custom_category],
        outputs=add_output
    )
    with gr.Column():
      gr.Markdown("### 📁 Import Your Subscription CSV")
      csv_upload = gr.File(label="Upload Your Subscription CSV", file_types=[".csv"])
      upload_status = gr.Textbox(label="Upload Status", lines=2)
      gr.Button("Import CSV").click(
        fn=import_manual_csv,
        inputs=[csv_upload],
        outputs=[upload_status]
)


    # 💸 Spending Breakdown
    spending_output = gr.Textbox(label="Spending Breakdown", lines=8)

    gr.Button("Calculate Spending").click(
      fn=lambda *args: calculate_spending(collect_selected(*args)),
      inputs=list(checkbox_groups.values()),
      outputs=spending_output
)

    # AI
    with gr.Blocks(theme=sparkle_theme) as demo:
      gr.Markdown("### 💬 Ask Budget Babe for Advice")

      chat_input = gr.Textbox(label="Ask Budget Babe", placeholder="e.g. What should I cut?")
      chatbot_output = gr.Chatbot(label="Budget Babe Advice", type="messages")
      chat_history = gr.State([])

      gr.Button("Ask Budget Babe AI").click(
        fn=budget_babe_chatbot,
        inputs=[chat_input, chat_history] + list(checkbox_groups.values()),
        outputs=[chatbot_output]
    )

    with gr.Column():
    # 🎯 Savings Progress Section
        gr.Markdown("### 🎯 Budget Babe Savings Tracker")

    with gr.Row():
        progress_bar = gr.Slider(
            minimum=0,
            maximum=100,
            label="Savings Progress",
            interactive=False,
            value=0
        )
        badge_output = gr.Markdown()

        gr.Button("Check Savings Progress").click(
          fn=update_savings_progress,
          inputs=[],  # ✅ No checkbox group here
          outputs=[progress_bar, badge_output])

    # 📋 Comparison Table
    with gr.Group():
      gr.Markdown("### 📋 Compare Your Subscriptions")

      comparison_output = gr.Markdown()

      gr.Button("Show Comparison Table").click(
        fn=lambda *selected_services_groups: compare_services_table(collect_selected(*selected_services_groups)),
        inputs=list(checkbox_groups.values()),
        outputs=[comparison_output]
    )


    app.launch()

Hiii there, Budget Babe! I'm so glad you're chatting with me today. Are you looking for some money-saving tips or just want to bounce ideas off me?
Enter your API key: ··········
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2dea76256d5c5ed0ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**STEP 6:** Test with a Variety of Data
Ensure:
- Accurate detection and - categorization
- Alerts trigger correctly
- Outputs are readable and business-ready

In [ ]:
print("🔍 COMPREHENSIVE TESTING SUITE")
print("=" * 40)

import pandas as pd

# Sample test dataset
def create_test_datasets():
    global test_df
    test_df = pd.DataFrame([
        {"Date": "2025-10-01", "Amount": 14.99, "Description": "Netflix", "Category": "Movies & TV"},
        {"Date": "2025-10-01", "Amount": 12.99, "Description": "Spotify", "Category": "Music"},
        {"Date": "2025-10-01", "Amount": 29.99, "Description": "HelloFresh", "Category": "Food"},
        {"Date": "2025-10-01", "Amount": 9.99, "Description": "Audible", "Category": "Reading"},
    ])
    assert not test_df.empty, "Test dataset is empty"
    print("✅ Test dataset created")

# Simulate loading function
def test_data_loading_function():
    assert "Description" in test_df.columns, "Missing 'Description' column"
    assert pd.api.types.is_numeric_dtype(test_df["Amount"]), "'Amount' column is not numeric"
    print("✅ Data loading function passed")

# Simulate spending analysis
def test_spending_analysis():
    monthly_total = test_df["Amount"].sum()
    assert monthly_total > 0, "Monthly total should be greater than zero"
    assert monthly_total == 67.96, f"Unexpected monthly total: {monthly_total}"
    print("✅ Spending analysis passed")

# Simulate business insights
def test_business_insights():
    food_spend = test_df[test_df["Category"] == "Food"]["Amount"].sum()
    assert food_spend > 25, "Food spend should trigger cut suggestion"
    overlapping_services = ["Netflix", "Disney+", "Stan"]
    active = [s for s in overlapping_services if s in test_df["Description"].values]
    assert len(active) < 3, "Too many streaming services detected"
    print("✅ Business insights passed")

# Run the suite
try:
    create_test_datasets()
    test_data_loading_function()
    test_spending_analysis()
    test_business_insights()
    print("🎉 All tests passed! Your finance assistant is working correctly.")
except AssertionError as e:
    print(f"❌ Test failed: {e}")
except Exception as e:
    print(f"⚠️ Test error: {e}")


🔍 COMPREHENSIVE TESTING SUITE
✅ Test dataset created
✅ Data loading function passed
✅ Spending analysis passed
✅ Business insights passed
🎉 All tests passed! Your finance assistant is working correctly.
